# Avoiding Overfitting

The ultimate goal of machine learning is to make accurate predictions on unseen data. EvalML aims to help you build a model that will perform as you expect once it is deployed in to the real world.

One of the benefits of using EvalML to build models is that it provides guardrails to ensure you are building pipelines that will perform reliably in the future. This page describes the various ways EvalML helps you avoid overfitting to your data.

In [1]:
import evalml

## Detecting Label Leakage

A common problem is having features that include information from your label in your training data. By default, EvalML will provide a warning when it detects this may be the case.

Let's set up a simple example to demonstrate what this looks like

In [2]:
import pandas as pd

X = pd.DataFrame({
    "leaked_feature": [6, 6, 10, 5, 5, 11, 5, 10, 11, 4],
    "leaked_feature_2": [3, 2.5, 5, 2.5, 3, 5.5, 2, 5, 5.5, 2],
    "valid_feature": [3, 1, 3, 2, 4, 6, 1, 3, 3, 11]
})

y = pd.Series([1, 1, 0, 1, 1, 0, 1, 0, 0, 1])

clf = evalml.AutoClassifier(
    max_pipelines=1,
    model_types=["linear_model"],
)

clf.fit(X, y)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Precision. Greater score is better.

Searching up to 1 pipelines. No time limit is set. Set one using max_time parameter.

Possible model types: linear_model

Testing LogisticRegression w/ imputation + scaling: 100%|██████████| 1/1 [00:01<00:00,  1.69s/it]

✔ Optimization finished


In the example above, EvalML warned about the input features `leaked_feature` and `leak_feature_2`, which are both very closely correlated with the label we are trying to predict. If you'd like to turn this check off, set `detect_label_leakage=False`.

The second way to find features that may be leaking label information is to look at the top features of the model. As we can see below, the top features in our model are the 2 leaked features.

In [3]:
best_pipeline = clf.best_pipeline
best_pipeline.feature_importances

,feature,importance
0,leaked_feature,-1.773115
1,leaked_feature_2,-1.731261
2,valid_feature,-0.247665



## Perform cross-validation for pipeline evaluation

By default, EvalML performs 3-fold cross validation when building pipelines. This means that it evaluates each pipeline 3 times using different sets of data for training and testing. In each trial, the data used for testing has no overlap from the data used for training.

While this is a good baseline approach, you can pass your own cross validation object to be used during modeling. The cross validation object can be any of the CV methods defined in [scikit-learn](https://scikit-learn.org/stable/modules/cross_validation.html) or use a compatible API.

For example, if we wanted to do a time series split:

In [4]:
from sklearn.model_selection import TimeSeriesSplit

X, y = evalml.demos.load_breast_cancer()

clf = evalml.AutoClassifier(
    cv=TimeSeriesSplit(n_splits=6), 
    max_pipelines=1
)

clf.fit(X, y)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Precision. Greater score is better.

Searching up to 1 pipelines. No time limit is set. Set one using max_time parameter.

Possible model types: linear_model, xgboost, random_forest

Testing Random Forest w/ imputation: 100%|██████████| 1/1 [00:16<00:00, 16.81s/it]

✔ Optimization finished


if we describe the 1 pipeline we built, we can see the scores for each of the 6 splits as determined by the cross-validation object we provided. We can also see the number of training examples per fold increased because we were using `TimeSeriesSplit`

In [5]:
clf.describe_pipeline(0)

************************
* Pipeline Description *
************************

Pipeline Name: Random Forest w/ imputation
Model type: random_forest
Objective: Precision (greater is better)
Total training time (including CV): 16.8 seconds

Parameters
• n_estimators: 569
• max_depth: 22
• impute_strategy: most_frequent
• percent_features: 0.8593661614465293

Cross Validation
               F1  Precision  Recall   AUC  Log Loss # Training # Testing
0           0.875      0.955   0.875 0.964     0.386     83.000    81.000
1           0.988      1.000   0.988 1.000     0.125    164.000    81.000
2           0.982      0.964   0.982 0.987     0.104    245.000    81.000
3           0.956      0.982   0.956 0.996     0.103    326.000    81.000
4           0.976      0.984   0.976 0.997     0.104    407.000    81.000
5           0.992      1.000   0.992 0.999     0.087    488.000    81.000
mean        0.961      0.981   0.961 0.991     0.152          -         -
std         0.044      0.019   0.04

## Detect unstable pipelines

When we perform cross validation we are trying generate an estimate of pipeline performance. EvalML does this by taking the mean of the score across the folds. If the performance across the folds varies greatly, it is indicative the the estimated value may be unreliable. 

To protect the user against this, EvalML checks to see if the pipeline's performance has a variance between the different folds. EvalML triggers a warning if the "coefficient of variance" of the scores (the standard deviation divided by mean) of the pipelines scores exeeds .2.

This warning will appear in the pipeline rankings under `high_variance_cv`.

In [6]:
clf.rankings

,id,pipeline_name,score,high_variance_cv,parameters
0,0,RFClassificationPipeline,0.980796,False,"{'n_estimators': 569, 'max_depth': 22, 'impute..."


## Create holdout for model validation

EvalML offers a method to quickly create an holdout validation set. A holdout validation set is data that is not used during the process of optimizing or training the model. You should only use this validation set once you've picked the final model you'd like to use.

Below we create a holdout set of 20% of our data

In [7]:
X, y = evalml.demos.load_breast_cancer()
X_train, X_holdout, y_train, y_holdout = evalml.preprocessing.split_data(X, y, test_size=.2)

In [8]:
clf = evalml.AutoClassifier(
    objective="recall",
    max_pipelines=3,
    detect_label_leakage=True
)
clf.fit(X_train, y_train)

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Recall. Greater score is better.

Searching up to 3 pipelines. No time limit is set. Set one using max_time parameter.

Possible model types: linear_model, xgboost, random_forest

Testing Random Forest w/ imputation: 100%|██████████| 3/3 [00:14<00:00,  4.74s/it]               

✔ Optimization finished


then we can retrain the best pipeline on all of our training data and see how it performs compared to the estimate

In [9]:
pipeline = clf.best_pipeline
pipeline.fit(X_train, y_train)
pipeline.score(X_holdout, y_holdout)

0.9861111111111112